In [1]:
import os
import pandas as pd
import io

In [2]:
root_files = os.listdir('trees_all')
sub_files_to_flatten = [[f'{r}/{f}' for f in os.listdir(f'trees_all/{r}')] for r in root_files]
#Flatten list of files into one list
sub_files = [f'trees_all/{item}' for sublist in sub_files_to_flatten for item in sublist]

In [3]:
# Trying to look at file structure
# Total number of files
print(len(sub_files))
# Total number of files with the -noaa extension
print(len([sf for sf in sub_files if ('-noaa' in sf)]))
# Total number of files without the -noaa extension
print(len([sf for sf in sub_files if not ('-noaa' in sf)]))

9644
4775
4869


In [4]:
xt_set = {sf[:-9] for sf in sub_files if ('-noaa' in sf)}
normal_set = {sf[:-4] for sf in sub_files if not ('-noaa' in sf)}
# Number of files without matching sets of -noaa and non -noaa files
print(len(xt_set ^ normal_set))
# Files without matching sets
print(xt_set ^ normal_set)

110
{'trees_all/europe/norw029', 'trees_all/europe/finl084', 'trees_all/europe/norw024', 'trees_all/north_america/wi058', 'trees_all/north_america/wi021', 'trees_all/north_america/tx006', 'trees_all/europe/finl083', 'trees_all/north_america/cana636', 'trees_all/north_america/ok043', 'trees_all/north_america/wi053', 'trees_all/europe/finl079', 'trees_all/north_america/ia036', 'trees_all/north_america/wi040', 'trees_all/north_america/wi029', 'trees_all/north_america/wi044', 'trees_all/europe/finl097', 'trees_all/europe/norw019', 'trees_all/europe/norw022', 'trees_all/europe/finl090', 'trees_all/north_america/wi060', 'trees_all/north_america/wi019', 'trees_all/europe/finl078', 'trees_all/north_america/wi033', 'trees_all/north_america/wi061', 'trees_all/north_america/va021original', 'trees_all/north_america/wi034', 'trees_all/europe/finl086', 'trees_all/north_america/ca727', 'trees_all/north_america/wi022', 'trees_all/north_america/wi050', 'trees_all/north_america/wi052', 'trees_all/north_

Seems for the most part, the files come in pairs and follow the format: {first 4 letters of country}{number}.rwl and {first 4 letters of country}{number}-noaa.rwl
- Files w/o -noaa ext: first 3 lines have info on the tree, + other metadata
- Files with -noaa ext: Many lines of text starting with "#" containing info about the data, then a line with column labels, the first of which is usually something with age

In [5]:
# Example of file with -noaa extension
print(sub_files[0])
file1 = open(sub_files[0],'r')
print(file1.read())
file1.close()

trees_all/africa/alge001-noaa.rwl
# Touchan - Leid Mohamad Ouali - CDAT - ITRDB ALGE001
#-----------------------------------------------------------------------
#               World Data Service for Paleoclimatology, Boulder
#                                  and
#                     NOAA Paleoclimatology Program
#             National Centers for Environmental Information (NCEI)
#-----------------------------------------------------------------------
# Template Version 3.0
# Encoding: UTF-8
# NOTE: Please cite original publication, online resource and date accessed when using these data,
# If there is no publication information, please cite investigator, title, online resource and date accessed.
#
# Online_Resource: https://www.ncdc.noaa.gov/paleo/study/9723
#             Description: NOAA Landing Page of study
# Online_Resource: https://www1.ncdc.noaa.gov/pub/data/paleo/treering/measurements/africa/alge001-noaa.rwl
#    Description: Raw Measurements - NOAA Template File
# Online_Re

In [6]:
# Example of file without -noaa extension
print(sub_files[1])
file1 = open(sub_files[1],'r')
print(file1.read())
file1.close()

trees_all/africa/alge001.rwl
LMO    1 Leid Mohamad Ouali                                  CDAT               
LMO    2 Algeria      Atlantic Cedar    1520M  3627 00406    __    1697 2005    
LMO    3 Ramzi Touchan                                                          
LMO01A  1822    59    31    44    56   101   110   109   200
LMO01A  1830   225   167   191   260   256   289   269    90    43    74
LMO01A  1840    48    32    14    32    71   137   153   173   150   164
LMO01A  1850   216   197   165   141   182    94    85   137     0    14
LMO01A  1860    14    26    52    93   130   168   122    61   130   152
LMO01A  1870   155   165   164   203   213   142   172   176   202   188
LMO01A  1880   232   172   100    22    47    81   111   141   219   255
LMO01A  1890   295   314   336   330   302   312   189   135   181   247
LMO01A  1900   183   214   186   185   190   270   279   278   317   316
LMO01A  1910   291   274   312   350   316   327   382   312   276   243
LMO01A  19

Hopefully not having matching sets isn't gonna be a problem if we just take all the data from the files with the -nooa extension and put it in one dataframe, and then take all the files with the extension and put that in a separate dataframe and find some way to merge them(maybe drop the ones without matching pairs?), but it would be good to know what that data actually is. 

In [7]:
#Attempt at cleaning a file w/ noaa xt
age_line = 0
with open(sub_files[0],'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

115
age_CE	LMO01A_raw	LMO01B_raw	LMO02A_raw	LMO02B_raw	LMO04A_raw	LMO05A_raw	LMO05B_raw	LMO06A_raw	LMO06B_raw	LMO07A_raw	LMO07B_raw	LMO08A_raw	LMO08B_raw	LMO09B_raw	LMO10A_raw	LMO10B_raw	LMO11A_raw	LMO11B_raw	LMO12A_raw	LMO13A_raw	LMO14A_raw	LMO14B_raw	LMO15A_raw	LMO16A_raw	LMO16B_raw	LMO17A_raw	LMO17B_raw	LMO18A_raw	LMO18B_raw	LMO19A_raw	LMO19B_raw	LMO20A_raw	LMO20B_raw	
 1697 	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	237	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	
 1698	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	183	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	
 1699	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	193	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	
 1700	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	192	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	
 1701	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	Na

In [8]:
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

,age_CE,LMO01A_raw,LMO01B_raw,LMO02A_raw,LMO02B_raw,LMO04A_raw,LMO05A_raw,LMO05B_raw,LMO06A_raw,LMO06B_raw,...,LMO16B_raw,LMO17A_raw,LMO17B_raw,LMO18A_raw,LMO18B_raw,LMO19A_raw,LMO19B_raw,LMO20A_raw,LMO20B_raw,Unnamed: 34
0,1697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Cleaning one without
with open(sub_files[1],'r') as f2:
    lines2 = f2.readlines()
d = [[w.strip() for w in textline.split(' ') if (w != '')] for textline in lines2[3:]]
# No idea what the columns should be
df2 = pd.DataFrame(d)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,LMO01A,1822,59,31,44,56,101,110,109,200,None,None
1,LMO01A,1830,225,167,191,260,256,289,269,90,43,74
2,LMO01A,1840,48,32,14,32,71,137,153,173,150,164
3,LMO01A,1850,216,197,165,141,182,94,85,137,0,14
4,LMO01A,1860,14,26,52,93,130,168,122,61,130,152
